In [ ]:
%%capture

import warnings

import _section1_utils as section1
import _section2_utils as section2
import calitp_data_analysis.magics
import great_tables as gt
import pandas as pd
from IPython.display import HTML, Image, Markdown, display, display_html
from shared_utils import catalog_utils, rt_dates, rt_utils

warnings.filterwarnings("ignore")

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
# Comment out and leave this cell right below pandas
# organization_name = "Peninsula Corridor Joint Powers Board"
# organization_name = "City and County of San Francisco"

In [ ]:
%%capture_parameters
organization_name

In [ ]:
most_recent_date = pd.to_datetime(rt_dates.DATES["mar2024"])
year = most_recent_date.year

In [ ]:
name = section1.organization_name_crosswalk(organization_name)

In [ ]:
# Load first dataset to grab the column "name"
operator_profiles = section1.load_operator_profiles(organization_name)

In [ ]:
# Datasets for Section 1
operator_route_map = section1.load_operator_map(name)
monthly_scheduled_service = section1.summarize_monthly(name)

In [ ]:
# Datasets for Section 2
sched_vp_df = section2.load_schedule_vp_metrics(name)
gtfs_schedule_keys = list(sched_vp_df.schedule_gtfs_dataset_key.unique())

In [ ]:
display(
    HTML(
        """
<style>
form.vega-bindings {
  position: absolute;
  right: 0px;
  top: 0px;
}
</style>
"""
    )
)

# {organization_name}

In [ ]:
display(
    Markdown(
        f"""<h2 style="border-bottom: 3px solid #8CBCCB;"><b>Operator Overview</b></h2>
               
        """
    )
)

In [ ]:
try:
    date = operator_profiles["service_date"].dt.to_pydatetime()[0]
except:
    pass

In [ ]:
try:
    display(
        Markdown(
            f"""{organization_name} ran <b>{operator_profiles.operator_n_routes.values[0]}</b> 
        unique routes on {date.month}/{date.year}. Below is the breakdown of the routes."""
        )
    )
except:
    
    display(
        Markdown(
            f"""{organization_name} doesn't have an operator profile."""
        )
    )

In [ ]:
try:
    section1.basic_pie_chart(
        section1.route_typology(operator_profiles),
        "route_type",
        "total_routes",
        "Total Routes by Typology",
    )
except:
    pass


In [ ]:
try:
    display(
    Markdown(
        f"""{organization_name} provided <b>{operator_profiles.operator_route_length_miles.values[0]}</b> miles of public transit.
        The average length of a route is <b>{operator_route_map.route_length_miles.mean()}</b> miles.
        """
    )
    )
except:
    pass

In [ ]:
try:
    section1.basic_bar_chart(
    section1.shortest_longest_route(operator_route_map),
    "route_length_miles",
    "route_combined_name",
    f"Longest and Shortest Route"
    )
except:
    pass

In [ ]:
display(
    Markdown(f"{organization_name} provides public transit in the following counties:")
)

In [ ]:
try:
    section1.counties_served(operator_route_map)
except:
    display(
        Markdown(
            f"""{organization_name} doesn't have an route geographies."""
        )
    )
    

In [ ]:
monthly_scheduled_service_df_agg = section1.summarize_monthly(name)

In [ ]:
section1.single_bar_chart_dropdown(
    monthly_scheduled_service_df_agg,
    "day_type",
    "ttl_service_hours",
    "time_of_day",
    "Total Service Hours per Month",
    "full_date",
)

In [ ]:
display(
    Markdown(
        f"""<h2 style="border-bottom: 3px solid #8CBCCB;"><b>Route Summary Statistics for {most_recent_date.month}/{most_recent_date.year}</b></h2>
               
        """
    )
)

In [ ]:
display(
    Markdown(
        f"""<h2 style="border-bottom: 3px solid #8CBCCB;"><b>Detailed Route-Direction Overview</b></h2><br>
               
        """
    )
)

In [ ]:
section2.trips_by_gtfs(sched_vp_df)

In [ ]:
display(
    Markdown(
        f"""Statistics presented on the <b>route</b> level. Scroll down to filter for a specific route. 
           """
    )
)

In [ ]:
section2.filtered_route(sched_vp_df)